<font size="+3"><strong>Exploring the Data</strong></font>

In this project we're going to work with data from the [Funda](https://funda.nl). This company provides houses pricing in the Netherlands.

Keep in mind that **Funda** prices are just a starting point and properties usually sold for 110% of their price or even more than 120% due to the auction nature.
During this auction, you should not know bids of others, but this is not true, because agents sometimes share current bids with their clients (this is not legal, but happens).
In addition, lower bid can win, because the person is not going to take out any loans (cash is ready to be transferred).

Additional expenses usually around 7%, but this is just additional information about property market in the Netherlands.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Prepare data

First, we need to load data. At this pount you already should have dump of data from [Funda](https://funda.nl) (if not, please follow the README file of this project).

In [ ]:
df = pd.read_json('../data/funda_15_08_2022.json')
df.info()
df.head()

All fields are dumped as a string. It is necessary to convert fields like **price**, **year_of_construction** and **living_area**.

## Field "price"

This field has four different states:
* start price is known (example: "€ 894,000 kosten koper")
* start price is known and tax included (example: "€ 1,442,000 op naam") 
* tender
* by request

In [ ]:
total_records = df['price'].count()
print(f'Total records {total_records}')

known_price_mask = df['price'].str.contains(pat='€ \d+[\d,]*\d+ kosten koper', regex=True) == True
known_price_count = df[known_price_mask]['price'].count()
print(f'Records with price: {known_price_count}')

known_tax_included_price_mask = df['price'].str.contains(pat='€ \d+[\d,]*\d+ vrij op naam', regex=True) == True
known_tax_included_price_count = df[known_tax_included_price_mask]['price'].count()
print(f'Records with price (tax included): {known_tax_included_price_count}')

price_on_request_mask = df['price'].str.contains('Price on request') == True
price_on_request_count = df[price_on_request_mask]['price'].count()
print(f'Records with prices on request: {price_on_request_count}')

sale_by_tender_mask = df['price'] == 'Sale by tender'
sale_by_tender_count = df[sale_by_tender_mask]['price'].count()
print(f'Records for tender: {sale_by_tender_count}')

missed_count = df[~known_price_mask & ~price_on_request_mask & ~known_tax_included_price_mask & ~sale_by_tender_mask]['price'].count()
assert missed_count == 0, 'Detected new type of records'

# plot
labels = ['Price without taxes', 'Price with taxes', 'Unknown starting bid']
values = [known_price_count, known_tax_included_price_count, price_on_request_count + sale_by_tender_count]
explode = (0, 0.1, 0.3)

fig1, ax1 = plt.subplots()
ax1.pie(values, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

## Field "year_of_construction"
This field contains information about cunstruction date of the property. Lets analyze its values.

In [ ]:
total_records = df['year_of_construction'].count()
print(f'Total records {total_records}')

known_year_mask = df['year_of_construction'].str.isdigit() == True
known_year_count = df[known_year_mask]['year_of_construction'].count()
print(f'Records with known year: {known_year_count}')

after_2020_year_mask = df['year_of_construction'] == 'After 2020'
after_2020_year_count = df[after_2020_year_mask]['year_of_construction'].count()
print(f'Records with known approximate date (after 2020): {after_2020_year_count}')

before_1906_year_mask = df['year_of_construction'] == 'Before 1906'
before_1906_year_count = df[before_1906_year_mask]['year_of_construction'].count()
print(f'Records with known approximate date (before): {before_1906_year_count}')

missed_count = df[~known_year_mask & ~after_2020_year_mask & ~before_1906_year_mask]['year_of_construction'].count()
assert missed_count == 0, 'Detected new type of records'

# plot
labels = ['Known year', 'Before 1906', 'After 2020']
values = [known_year_count, before_1906_year_count, after_2020_year_count]
explode = (0, 0.1, 0.3)

fig1, ax1 = plt.subplots()
ax1.pie(values, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

Interesting field values: **After 2020** and **Before 1906**. Sounds like there are three ranges: 

- before 1906
- in range from 1906 to 2022
- after 2022

Lets check, are there any dates before **1906** and after **2020**.

In [ ]:
df_years = df[known_year_mask]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=[20, 15])

# before 1906
known_years_before_1906_mask = df_years['year_of_construction'].astype(int) < 1906
known_years_before_1906 = df_years[known_years_before_1906_mask]['year_of_construction'].astype(int)
known_years_before_1906.plot.hist(bins=50, ax=axes[0]);

# after 2020, but found that construction date is not limited (they can sell properties that will be built centuries later)
known_years_after_2020_mask = (df_years['year_of_construction'].astype(int) > 2020) & (df_years['year_of_construction'].astype(int) < 2040)
known_years_after_2020 = df_years[known_years_after_2020_mask]['year_of_construction'].astype(int)
known_years_after_2020.plot.hist(bins=5, ax=axes[1]);

As result we see that around 2% of records has construction date **Before 1906** or **After 2020** (instead of a year). In addition, years are not limited (they are not doing sanity checks).

## Field "living_area"
This field contains property size in square meters.

In [ ]:
# prepare data to be integer
df_living_area = df['living_area'].str.replace(',', '').astype(int).to_frame()

# remove outliers by trimming the bottom and top 10% of properties
min, max = df_living_area['living_area'].quantile([0.1, 0.9])
mask_area = df_living_area['living_area'].between(min, max)
df_living_area = df_living_area[mask_area]
    
df_living_area.boxplot(figsize=[20, 5], vert=False, medianprops=dict(linewidth=2, color='r')).set_xlabel('Living Area (m2)');

According to the results, merian is more than 120 square meters. Further investigation can take into accound different areas, property types and year of construction.